In [1]:
import numpy as np
import networkx as nx
import pandas as pd
import heapq

0a) Download the same "wikispeedia" dataset from the PageRank homework assignment.

In [2]:
path = '/Users/maxperozek/CP341/Day4/wikispeedia_paths-and-graph/links.tsv'

links_df = pd.read_csv(path, sep='\t', on_bad_lines='skip')

In [3]:
edges = []
for index, row in links_df.iterrows():
    edges.append((row['to'],row['from']))

In [4]:
links_gr = nx.DiGraph()
links_gr.add_edges_from(edges)

In [5]:
import os

In [6]:
rootdir = '/Users/maxperozek/CP341/Day4/plaintext_articles/'

db = {}
for subdir, dirs, files in os.walk(rootdir):
    
    for file in files:
        with open(rootdir + file, "r") as f:
            text = f.read()
            wordlist = text.split()
            wordlist = set(wordlist)
            wordlist = list(wordlist)
            db[file[:-4]] = wordlist

In [7]:
len(db)

4604

In [8]:
path = '/Users/maxperozek/CP341/Day4/wikispeedia_paths-and-graph/articles.tsv'
idx_db = {}

idx_df = pd.read_csv(path, sep='\t', on_bad_lines='skip')

for index, row in idx_df.iterrows():
    idx_db[row['title']] = index

In [9]:
np_idx = idx_df.to_numpy().reshape((4604,
                               ))

In [10]:
np_idx.shape

(4604,)

In [11]:
np_idx[0]

'%C3%81ed%C3%A1n_mac_Gabr%C3%A1in'

In [12]:
idx_db['Ghana']

1696

Sample search:

Ghana -> Atlantic_Ocean -> Baltic_Sea ->  Telephone

Note: This may not be the shortest path, but this path does exist, so it should at least succeed when running search

0b) Implement Dijkstra's algorithm, AKA bag-search with a priority queue. You should use the heapq package in python (which is already installed) as the priority queue. As we discussed in class, it's important to keep track of which nodes have been visited, along with their predecessor along the shortest path.

In [13]:
# pops lowest value first!
# heapq.heappush(my_pqueue, (1.0, 'Dog'))

# for backtracking from destination all the way back to src
# backtracks FROM u TO v
def get_path(gr, u, v):
    path = []
    cur = u
    while cur:
        path.append(cur)
        cur = gr.nodes[cur]['prev']
    return list(reversed(path))
# don't forget to skip nodes we've already seen!!!
def dijkstras(gr, src, dst):
    gr = gr.copy()
    pq = []
    for node in gr:
        if node == src:
            gr.nodes[node]['dist'] = 0.0
            gr.nodes[node]['prev'] = None
            heapq.heappush(pq, (0.0, node))
        else:
            gr.nodes[node]['dist'] = np.inf
            gr.nodes[node]['prev'] = None
            heapq.heappush(pq, (np.inf, node))
            
    while pq:
        v = heapq.heappop(pq)[1]
        for neighbor in gr[v]:
            new_dst = gr.nodes[v]['dist'] + 1 # note I am assuming that the distance of following a link is cost(1)
            if new_dst < gr.nodes[neighbor]['dist']:
                gr.nodes[neighbor]['dist'] = new_dst
                gr.nodes[neighbor]['prev'] = v
                heapq.heappush(pq, (new_dst, neighbor))
        if v == dst:
            return get_path(gr, dst, src)


In [14]:
out_path = dijkstras(links_gr, 'Ghana', 'Telephone')

In [15]:
out_path

['Ghana', 'Atlantic_Ocean', 'Telephone']

1a) As part of the PageRank homework, we kept track of a dictionary of which terms occurred on each page. Modify your implementation of Dijkstra's to use a terminology overlap heuristic. So, the priority of a page in our queue will be given by:
If A and B are the two sets of words on the two pages, then a good starting heuristic to use is:
len( A.intersection(B) ) / len( A.union(B) )

In [16]:
from tqdm import tqdm

In [17]:
# pre-calculate heurisric matrix
def h(u,v):
    A = set(db[u])
    B = set(db[v])
    intersect_len = len(A.intersection(B))
    union_len = len(A.union(B))
    return intersect_len / union_len

def calc_heuristic():
    h_mat = -1 * np.ones((len(idx_df),len(idx_df)))

    pbar = tqdm(total = h_mat.shape[0] * (h_mat.shape[0] -1), position=0, leave=True)
    for i in range(h_mat.shape[0]):
        for j in range(h_mat.shape[0]):
            if(i != j and h_mat[j,i] == -1):
                h_mat[i,j] = h(np_idx[i], np_idx[j])
                pbar.update(1)
    
    h_mat_t = h_mat.T
    for i in range(h_mat.shape[0]):
        for j in range(h_mat.shape[0]):
            if(i != j and h_mat[j,i] != -1):
                h_mat[i,j] = h_mat_t[i,j]
                pbar.update(1)
    pbar.close()
    return h_mat

In [18]:
h_mat = calc_heuristic()

100%|█████████████████████████████| 21192212/21192212 [49:39<00:00, 7112.68it/s]


In [20]:
import torch

/Users/maxperozek/opt/anaconda3/envs/comp_gr_thy/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
torch.save(h_mat, 'h_mat.pt')

In [22]:
# h_mat = torch.load('h_mat.pt')

In [25]:
# def h(u,v):
#     A = set(db[u])
#     B = set(db[v])
#     intersect_len = len(A.intersection(B))
#     union_len = len(A.union(B))
#     return intersect_len / union_len

# for backtracking from destination all the way back to src
# backtracks FROM u TO v
def get_path(gr, u, v):
    path = []
    cur = u
    while cur:
        path.append(cur)
        cur = gr.nodes[cur]['prev']
    return list(reversed(path))
# don't forget to skip nodes we've already seen!!!
def astar(gr, src, dst):
    gr = gr.copy()
    pq = []
    for node in gr:
        if node == src:
            gr.nodes[node]['dist'] = 0.0
            gr.nodes[node]['prev'] = None
            heapq.heappush(pq, (0.0, node))
        else:
            gr.nodes[node]['dist'] = np.inf
            gr.nodes[node]['prev'] = None
            heapq.heappush(pq, (np.inf, node))
            
    while pq:
        v = heapq.heappop(pq)[1]
        for neighbor in gr[v]:
            # new_dst = gr.nodes[v]['dist'] + 1 + h(v, neighbor)
            new_dst = gr.nodes[v]['dist'] + 1 + h_mat[idx_db[v], idx_db[neighbor]]

            if new_dst < gr.nodes[neighbor]['dist']:
                gr.nodes[neighbor]['dist'] = new_dst
                gr.nodes[neighbor]['prev'] = v
                heapq.heappush(pq, (new_dst, neighbor))

        if v == dst:
            return get_path(gr, dst, src)


In [26]:
out_path = astar(links_gr, 'Ghana', 'Telephone')

In [27]:
out_path = astar(links_gr, 'Ghana', 'Atlantic_Ocean')

In [28]:
out_path

['Ghana', 'Atlantic_Ocean']

2a) the wikispeedia dataset contains a file of the shortest human-found paths between pairs of articles. Load these paths, and compare the lengths of a large number (~100) of paths to paths found by your methods in P0 and P1.

In [29]:
shortest_pathname = '/Users/maxperozek/CP341/Day4/wikispeedia_paths-and-graph/shortest-path-distance-matrix.txt'

path_matrix = -1 * np.ones((4604, 4604)) # -1 indicates that there is no path from the surce to the target

with open(shortest_pathname) as f:
    row = 0
    for line in f:
        if line[0] != '#' and line[0] != '\n':
            line = ''.join(c for c in line if( c.isalnum() or c == '_'))
            col = 0
            for char in line:
                # print(char)
                if char != '_':  
                    path_matrix[row,col] = char
                col += 1
            row += 1

In [30]:
path_matrix

array([[ 0., -1., -1., ...,  4.,  4.,  2.],
       [-1.,  0., -1., ...,  3.,  3.,  3.],
       [-1., -1.,  0., ...,  3.,  3.,  3.],
       ...,
       [-1., -1., -1., ...,  0.,  3.,  3.],
       [-1., -1., -1., ...,  4.,  0.,  3.],
       [-1., -1., -1., ...,  3.,  3.,  0.]])

In [31]:
path_matrix[0,1]

-1.0

In [32]:
np.random.seed(1)
rand_nodes = np.random.choice(links_gr.nodes, 200, replace=False)
srcs = rand_nodes[:100]
dsts = rand_nodes[100:]

In [33]:
i = 0
total = srcs.shape[0]
while i < total:
    if path_matrix[ idx_db[srcs[i]], idx_db[dsts[i]] ] == -1:
        srcs = np.delete(srcs, np.where(srcs == srcs[i]))
        dsts = np.delete(dsts, np.where(dsts == dsts[i]))
        total -= 1
    i += 1

In [34]:
print(srcs.shape, dsts.shape)

(89,) (89,)


In [35]:
paths = np.stack((srcs, dsts)).T

In [36]:
paths.shape

(89, 2)

In [38]:
len_human = []
len_dij = []
len_heur = []

pbar = tqdm(total=len(paths), position=0, leave=True)
for p in paths:
    len_human.append(path_matrix[ idx_db[p[0]], idx_db[p[1]] ])
    len_dij.append(len(dijkstras(links_gr, p[0], p[1])))
    len_heur.append(len(astar(links_gr, p[0], p[1])))
    pbar.update(1)
pbar.close()

57it [00:53,  1.07it/s]                                  | 0/89 [00:00<?, ?it/s]
 89%|██████████████████████████████████████▏    | 79/89 [01:14<00:07,  1.27it/s]
KeyboardInterrupt



Running got hung up on one of the last paths. I'm not really sure what's going on here... It never terminated on its own but the lists were still appended to through the point that I interrupted and are saved in memory, so I've observed the means below. My best guess is that memory is getting filled up with garbage throughout running this. I know that python does garbage collection for me but it's just a hunch, since I removed the paths that were unable to be found.

In [39]:
import statistics

In [40]:
statistics.mean(len_human)

3.125

In [41]:
statistics.mean(len_dij)

4.15

In [42]:
statistics.mean(len_heur)

4.151898734177215

The human found best path seems to outperform my 'dijkstras' implementation by 1 link on average. I am waiting until I can leave my computer for a handful of hours before I calculate the heuristic lookup table and run the full heuristic search.
One way that the heuristic could be improved would be making the calculation faster, perhaps processing the article text further to remove unimportant words and possibly even weight key words more heavily. Any way of decreasing the time of the expensive union and intersection operations.